In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fastai import *
from fastai.vision import *
from pathlib import Path
from fastai.callbacks.hooks import *
from cape_core.classifier import *

In [3]:
PATH = Path.cwd()
folder = 'zoom20_256'
IMAGEPATH = Path.cwd()/folder

In [4]:
data = get_data(path=PATH); data.head()

,index,p_ab_rs_band,p_90_rs_band,dont_use_rs,p_ab_cs_band,p_90_cs_band,dont_use_cs,p_ab_3party_band,p_90_3party_band,dont_use_3party,...,cad_id,prop_id,asatdate,prop_titledeedtype,estate_id,sgr_id,suburb_id,trf_purchprice,prop_munvalinfl,train
23072,220570,8,10,0,4,7,0,0,0,1,...,6905651,11944540,2017-04-15,FH,0.0,0,9834.0,1550000.0,0.0,True
54670,190005,8,9,0,6,8,0,0,0,1,...,4669236,11360646,2017-09-11,FH,0.0,0,3496.0,900000.0,0.0,True
8194,193975,8,8,0,7,10,0,0,0,1,...,1057686,11435648,2017-02-08,FH,0.0,0,48.0,1995000.0,0.0,True
67378,446903,9,10,0,7,8,0,0,0,1,...,5487873,15736748,2017-11-07,FH,0.0,0,5545.0,900000.0,0.0,True
62748,48275,8,8,0,6,8,0,0,0,1,...,168364,9485339,2017-10-17,FH,0.0,0,4339.0,1230000.0,939320.0,True


In [5]:
# for im in ['zoom18', 'zoom15']:
#     verify_images(Path.cwd()/im, max_workers=20, max_size=256, dest=PATH/(im+'_256'))

In [6]:
available_labels = get_available_images(IMAGEPATH)

In [7]:
available_labels.head()

,fname,cad_id
0,cad4651444.jpg,4651444
1,cad4432494.jpg,4432494
2,cad5597246.jpg,5597246
3,cad1776714.jpg,1776714
4,cad5587051.jpg,5587051


In [8]:
all_ids, top10_ids = get_classifier_df(data, available_labels)

In [9]:
all_ids.head()

,cad_id,trf_purchprice,train,fname,top10
0,6905651,1550000.0,True,cad6905651.jpg,NaN
1,4669236,900000.0,True,cad4669236.jpg,NaN
2,1057686,1995000.0,True,cad1057686.jpg,NaN
3,5487873,900000.0,True,cad5487873.jpg,NaN
4,168364,1230000.0,True,cad168364.jpg,NaN


In [10]:
val_idxs = top10_ids[~top10_ids['train']].index

In [11]:
db = get_db(df=top10_ids, test_df=all_ids, val_idxs=val_idxs, path=PATH, folder=folder, bs=128)

## Model

In [40]:
n_features = 256

learn = classification_learner(db, 
                               models.resnet34,
                               pretrained=True,
                               head = create_head(512, 2, lin_ftrs=[n_features], concat_pool=False),
                               wd=1).mixup().to_fp16()

In [34]:
learn.model[1][0:2]

Sequential(
  (0): AdaptiveAvgPool2d(output_size=1)
  (1): Flatten()
)

In [14]:
# learn.lr_find(); learn.recorder.plot()

In [15]:
learn.fit_one_cycle(20, 1e-3)
learn.save('r34_sample_'+folder, return_path=True)

epoch,train_loss,valid_loss,error_rate,time
0,0.557198,0.412311,0.197121,00:48
1,0.495882,0.385478,0.183555,00:47
2,0.453846,0.425047,0.208749,00:47
3,0.430482,0.335355,0.161406,00:47
4,0.419802,0.322289,0.141750,00:46
5,0.412605,0.591677,0.323090,00:47
6,0.397712,0.509223,0.239480,00:47
7,0.384949,0.266317,0.108250,00:47
8,0.372486,0.250837,0.099114,00:46
9,0.360881,0.250896,0.101606,00:47


PosixPath('/home/tc256760/Documents/Tabular_image_model/Sample Models/models/xr34_sample_zoom20_256.pth')

## Create Features

In [16]:
learn.load('xr34_sample_'+folder);

In [17]:
features = create_features(learn, all_ids, PATH, head_layers=slice(0,2))

In [18]:
features.head()

,cad_id,img_feat_0,img_feat_1,img_feat_2,img_feat_3,img_feat_4,img_feat_5,img_feat_6,img_feat_7,img_feat_8,...,img_feat_502,img_feat_503,img_feat_504,img_feat_505,img_feat_506,img_feat_507,img_feat_508,img_feat_509,img_feat_510,img_feat_511
0,6905651,0.001901,0.002313,0.001925,0.001879,0.002021,0.002060,0.001974,0.001899,0.001903,...,0.001842,0.001861,0.001839,0.001835,0.001906,0.002033,0.002133,0.002022,0.001816,0.002023
1,4669236,0.001920,0.001929,0.001853,0.001910,0.001936,0.001910,0.001987,0.001880,0.001868,...,0.002043,0.001905,0.001919,0.001877,0.002092,0.001969,0.001947,0.001923,0.001913,0.002003
2,1057686,0.001940,0.002048,0.001887,0.001839,0.001859,0.001857,0.001838,0.002063,0.002000,...,0.001916,0.001992,0.001907,0.001758,0.001846,0.002151,0.001788,0.001822,0.001896,0.002234
3,5487873,0.001948,0.002045,0.001929,0.001868,0.001986,0.001939,0.001892,0.001911,0.001846,...,0.001883,0.001946,0.001894,0.001859,0.001909,0.002114,0.001900,0.001947,0.001834,0.002085
4,168364,0.001907,0.002008,0.001840,0.001897,0.001934,0.001881,0.001999,0.001865,0.001902,...,0.002018,0.001915,0.001865,0.001881,0.002025,0.001957,0.001981,0.001904,0.001881,0.001973


In [19]:
features.to_pickle('feat_xr34_512_' + folder)